In [1]:
import fitz  # PyMuPDF

In [20]:
file_path = "/Users/aditkotwal/Projects/FinancialAnalysisApp/Backend/aapl_10-K_report.pdf"
pdf_path = "/Users/aditkotwal/Projects/FinancialAnalysisApp/Backend/aapl_10-K_report.pdf"

In [16]:
document = fitz.open(file_path)
headings = []
page = document[10]
# blocks = page.get_text()
blocks = page.get_text("dict")["blocks"]

In [19]:
blocks[0]["lines"]

[{'spans': [{'size': 9.0,
    'flags': 20,
    'char_flags': 16,
    'font': 'Helvetica-Bold',
    'color': -16777216,
    'ascender': 0.7699999809265137,
    'descender': -0.2290000021457672,
    'text': 'Business Seasonality and Product Introductions',
    'origin': (62.25, 127.693359375),
    'bbox': (62.25,
     120.75642395019531,
     269.7989807128906,
     129.7564239501953)}],
  'wmode': 0,
  'dir': (1.0, 0.0),
  'bbox': (62.25, 120.75642395019531, 269.7989807128906, 129.7564239501953)}]

In [26]:
def extract_headings_from_pdf(file_path):
    # Open the PDF file
    document = fitz.open(file_path)
    headings = []

    for page_num in range(len(document)):
        page = document[page_num]
        blocks = page.get_text("dict")["blocks"]
        
        for block in blocks:
            if "lines" in block:
                for line in block["lines"]:
                    for span in line["spans"]:
                        # Customize the font size threshold as needed
                        if span["size"] >= 8:  # Assuming headings have a larger font size
                            headings.append(span["text"])

    document.close()
    return headings


headings = extract_headings_from_pdf(pdf_path)

In [27]:
headings

['UNITED STATES',
 'SECURITIES AND EXCHANGE COMMISSION',
 'Washington, D.C. 20549',
 'FORM 10-K',
 '(Mark One)',
 '☒',
 'ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934',
 'For the fiscal year ended September\xa028, 2024',
 'or',
 '☐',
 'TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF',
 '1934',
 'For the transition period from ',
 ' to',
 ' ',
 '.',
 'Commission File Number: ',
 '001-36743',
 'Apple Inc.',
 '(Exact name of Registrant as specified in its charter)',
 'California',
 '94-2404110',
 '(State or other jurisdiction',
 'of incorporation or organization)',
 '(I.R.S. Employer Identification No.)',
 'One Apple Park Way',
 'Cupertino, California',
 '95014',
 '(Address of principal executive offices)',
 '(Zip Code)',
 '(408) 996-1010',
 '(Registrant’s telephone number, including area code)',
 'Securities registered pursuant to Section 12(b) of the Act:',
 'Title of each class',
 'Trading',
 'symbol(s)',
 'Name

In [31]:
"Risk Factors" in headings

True